In [1]:
import glob
import os

import pandas as pd
import seaborn as sns

In [7]:
pd.set_option("display.max_columns", None)
pd.set_option('display.max_rows', 100)
pd.options.display.float_format = '{:.3f}'.format

#### Ta reda på antalet kolumner och rader i alla områden:

In [3]:
%%time
df_list = []
nr_of_files = 0
for subdir, dirs, files in os.walk("../data/areas_of_interest/"):
    for file in files:
        if file.endswith(".gzip"):
            nr_of_files += 1
            file_path = os.path.join(subdir, file)
            df = pd.read_parquet(file_path)
            df_list.append(df)
print(nr_of_files)
print(len(df_list))

1793
1793
CPU times: user 21.8 s, sys: 5.38 s, total: 27.2 s
Wall time: 14.1 s


#### One single dataframe for all listings:

In [4]:
all_df_files_concat = pd.concat(df_list)
all_df_files_concat.shape

(112703, 46)

In [8]:
all_df_files_concat.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 112703 entries, 0 to 7
Data columns (total 46 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   id                               112703 non-null  int64  
 1   coordinate                       112703 non-null  object 
 2   address                          112703 non-null  object 
 3   location_name                    112703 non-null  object 
 4   typeSummary                      112703 non-null  object 
 5   iconName                         112703 non-null  object 
 6   project                          112703 non-null  bool   
 7   new_construction                 112703 non-null  bool   
 8   ongoing_bidding                  112703 non-null  bool   
 9   foreclosure                      112703 non-null  bool   
 10  upcoming                         112703 non-null  bool   
 11  price                            112661 non-null  object 
 12  price_p

In [6]:
all_df_files_concat["id"].nunique()

4934

In [9]:
all_df_files_concat.sort_values(by=["id", "date"], ascending=[True, True], inplace=True)
all_df_files_concat.reset_index(inplace=True)
all_df_files_concat.drop(columns="index", inplace=True)
all_df_files_concat

,id,coordinate,address,location_name,typeSummary,iconName,project,new_construction,ongoing_bidding,foreclosure,upcoming,price,price_per_area,has_price_change,rooms,fee,age_string,age,deactivated_before_open_house,land_area,small_image_url,medium_image_url,url,labels,state_labels,user_saved,save_button_label,broker_logo_url,broker_agency_name,locations_string,should_display_showings,should_highlight_showings,living_space.table,trackingData.listingId,trackingData.listingType,trackingData.listingPackageType,trackingData.listingState,trackingData.listingStatus,broker_name,broker_phone,broker_email,price_change,area,date,visible_project_units,show_visible_project_units
0,15587995,"[59.3328524385163, 18.08369671727422]",Grevgatan 9,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,43 500 000 kr,195 946 kr/m²,False,6 rum,8 065 kr/mån,<span>89</span> dagar,89,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-6rum-ost...,"[{'category': 'feature', 'identifier': 'balcon...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Lagerlings,"Östermalm, Stockholms kommun",False,False,"{'living_area': '222 m²', 'supplemental_area':...",15587995,common_listing,not_selected,active,for_sale,None,None,None,NaN,ostermalm,20221109,NaN,NaN
1,16090535,"[59.3383624783, 18.0137004511]",Brf Topaz,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,True,True,False,False,False,fr. 2 854 500 kr,fr. 109 855 kr/m²,False,1-3 rum,fr. 1 667 kr/mån,<span>1</span> dagar,1,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/nybyggnadsprojekt/vastra...,"[{'category': 'type', 'hidden': False, 'identi...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Fastighetsbyrån Stockholm - Kungsholmen,"Västra Kungsholmen, Stockholms kommun",False,False,"{'living_area': '24–67 m²', 'supplemental_area...",16090535,project,not_selected,active,for_sale,None,None,None,NaN,kungsholmen,20230128,5.000,False
2,16090535,"[59.3383624783, 18.0137004511]",Brf Topaz,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,True,True,False,False,False,fr. 2 854 500 kr,fr. 109 855 kr/m²,False,1-3 rum,fr. 1 667 kr/mån,<span>2</span> dagar,2,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/nybyggnadsprojekt/vastra...,"[{'category': 'type', 'hidden': False, 'identi...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Fastighetsbyrån Stockholm - Kungsholmen,"Västra Kungsholmen, Stockholms kommun",False,False,"{'living_area': '24–67 m²', 'supplemental_area...",16090535,project,not_selected,active,for_sale,None,None,None,NaN,kungsholmen,20230129,5.000,False
3,16090535,"[59.3383624783, 18.0137004511]",Brf Topaz,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,True,True,False,False,False,fr. 2 854 500 kr,fr. 106 700 kr/m²,False,1-3 rum,fr. 1 667 kr/mån,<span>3</span> dagar,3,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/nybyggnadsprojekt/vastra...,"[{'category': 'type', 'hidden': False, 'identi...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Fastighetsbyrån Stockholm - Kungsholmen,"Västra Kungsholmen, Stockholms kommun",False,False,"{'living_area': '24–67 m²', 'supplemental_area...",16090535,project,not_selected,active,for_sale,None,None,None,NaN,kungsholmen,20230130,5.000,False
4,16090535,"[59.3383624783, 18.0137004511]",Brf Topaz,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,True,True,False,False,False,fr. 3 227 321 kr,fr. 104 107 kr/m²,False,1-3 rum,fr. 1 864 kr/mån,<span>4</span> dagar,4,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/nybyggnadsprojekt/vastra...,"[{'category'

In [10]:
listings_without_dates = all_df_files_concat.query("date.isna()").copy()
listings_without_dates.shape

(0, 46)

In [11]:
listings_without_dates["id"].nunique()

0

In [12]:
listings_without_dates["area"].value_counts()

Series([], Name: area, dtype: int64)

In [13]:
listings_without_dates

,id,coordinate,address,location_name,typeSummary,iconName,project,new_construction,ongoing_bidding,foreclosure,upcoming,price,price_per_area,has_price_change,rooms,fee,age_string,age,deactivated_before_open_house,land_area,small_image_url,medium_image_url,url,labels,state_labels,user_saved,save_button_label,broker_logo_url,broker_agency_name,locations_string,should_display_showings,should_highlight_showings,living_space.table,trackingData.listingId,trackingData.listingType,trackingData.listingPackageType,trackingData.listingState,trackingData.listingStatus,broker_name,broker_phone,broker_email,price_change,area,date,visible_project_units,show_visible_project_units


In [14]:
all_df_files_concat.query("id == 15587995")

,id,coordinate,address,location_name,typeSummary,iconName,project,new_construction,ongoing_bidding,foreclosure,upcoming,price,price_per_area,has_price_change,rooms,fee,age_string,age,deactivated_before_open_house,land_area,small_image_url,medium_image_url,url,labels,state_labels,user_saved,save_button_label,broker_logo_url,broker_agency_name,locations_string,should_display_showings,should_highlight_showings,living_space.table,trackingData.listingId,trackingData.listingType,trackingData.listingPackageType,trackingData.listingState,trackingData.listingStatus,broker_name,broker_phone,broker_email,price_change,area,date,visible_project_units,show_visible_project_units
0,15587995,"[59.3328524385163, 18.08369671727422]",Grevgatan 9,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,43 500 000 kr,195 946 kr/m²,False,6 rum,8 065 kr/mån,<span>89</span> dagar,89,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,https://www.hemnet.se/bostad/lagenhet-6rum-ost...,"[{'category': 'feature', 'identifier': 'balcon...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Lagerlings,"Östermalm, Stockholms kommun",False,False,"{'living_area': '222 m²', 'supplemental_area':...",15587995,common_listing,not_selected,active,for_sale,None,None,None,NaN,ostermalm,20221109,NaN,NaN


In [15]:
all_df_files_concat.query("id == 16430425")

,id,coordinate,address,location_name,typeSummary,iconName,project,new_construction,ongoing_bidding,foreclosure,upcoming,price,price_per_area,has_price_change,rooms,fee,age_string,age,deactivated_before_open_house,land_area,small_image_url,medium_image_url,url,labels,state_labels,user_saved,save_button_label,broker_logo_url,broker_agency_name,locations_string,should_display_showings,should_highlight_showings,living_space.table,trackingData.listingId,trackingData.listingType,trackingData.listingPackageType,trackingData.listingState,trackingData.listingStatus,broker_name,broker_phone,broker_email,price_change,area,date,visible_project_units,show_visible_project_units
86,16430425,"[59.3370827375, 18.0859475129]",Skeppargatan 54 A,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,16 500 000 kr,206 250 kr/m²,False,"2,5 rum",1 574 kr/mån,<span>348</span> dagar,348,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,"https://www.hemnet.se/bostad/lagenhet-2,5rum-o...","[{'category': 'product', 'identifier': 'plus',...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Fastighetsbyrån Stockholm – Östermalm,"Östermalm, Stockholms kommun",False,False,"{'living_area': '80', 'supplemental_area': '6 ...",16430425,common_listing,plus,active,for_sale,None,None,None,NaN,ostermalm,20221109,NaN,NaN
87,16430425,"[59.3370827375, 18.0859475129]",Skeppargatan 54 A,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,16 500 000 kr,206 250 kr/m²,False,"2,5 rum",1 574 kr/mån,<span>349</span> dagar,349,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,"https://www.hemnet.se/bostad/lagenhet-2,5rum-o...","[{'category': 'product', 'identifier': 'plus',...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Fastighetsbyrån Stockholm – Östermalm,"Östermalm, Stockholms kommun",False,False,"{'living_area': '80', 'supplemental_area': '6 ...",16430425,common_listing,plus,active,for_sale,None,None,None,NaN,ostermalm,20221110,NaN,NaN
88,16430425,"[59.3370827375, 18.0859475129]",Skeppargatan 54 A,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,16 500 000 kr,206 250 kr/m²,False,"2,5 rum",1 574 kr/mån,<span>350</span> dagar,350,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,"https://www.hemnet.se/bostad/lagenhet-2,5rum-o...","[{'category': 'product', 'identifier': 'plus',...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Fastighetsbyrån Stockholm – Östermalm,"Östermalm, Stockholms kommun",False,False,"{'living_area': '80', 'supplemental_area': '6 ...",16430425,common_listing,plus,active,for_sale,None,None,None,NaN,ostermalm,20221111,NaN,NaN
89,16430425,"[59.3370827375, 18.0859475129]",Skeppargatan 54 A,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,16 500 000 kr,206 250 kr/m²,False,"2,5 rum",1 574 kr/mån,<span>351</span> dagar,351,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,"https://www.hemnet.se/bostad/lagenhet-2,5rum-o...","[{'category': 'product', 'identifier': 'plus',...",[],False,Spara,https://bilder.hemnet.se/images/broker_logo_2/...,Fastighetsbyrån Stockholm – Östermalm,"Östermalm, Stockholms kommun",False,False,"{'living_area': '80', 'supplemental_area': '6 ...",16430425,common_listing,plus,active,for_sale,None,None,None,NaN,ostermalm,20221112,NaN,NaN
90,16430425,"[59.3370827375, 18.0859475129]",Skeppargatan 54 A,Stockholms kommun,Bostadsrättslägenhet,bostadsratt,False,False,False,False,False,16 500 000 kr,206 250 kr/m²,False,"2,5 rum",1 574 kr/mån,<span>352</span> dagar,352,False,None,https://bilder.hemnet.se/images/itemgallery_S/...,https://bilder.hemnet.se/images/itemgallery_M/...,"https://www.hemnet.se/bostad/lagenhet-2,5rum-o...","[{'cate

In [16]:
all_df_files_concat.drop_duplicates(subset="id", keep="first", ignore_index=True, inplace=True)
all_df_files_concat.shape

(4934, 46)

In [17]:
all_df_files_concat["date"].value_counts().reset_index().sort_values("index")

,index,date
0,20221109,1005
127,20221110,9
5,20221111,110
23,20221112,35
128,20221113,9
...,...,...
66,20230417,19
32,20230418,28
35,20230419,27
94,20230420,13
